In [1]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [2]:
import librosa
import soundfile
import numpy as np

# Function to extract features from audio file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
            result = np.hstack((result, mel))
    return result


In [3]:
#Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [4]:
print(observed_emotions)

['calm', 'happy', 'fearful', 'disgust']


In [5]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("C:\\Users\\NERAJAKSHA\\Desktop\\New folder\\ravdess data\\Actor_*\\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [6]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [7]:
print(x_train,x_test)

[[-5.22061890e+02  3.50668907e+01  3.75342965e+00 ...  1.65243138e-04
   1.04321596e-04  6.55571566e-05]
 [-6.41227722e+02  4.49487762e+01 -1.85174167e+00 ...  3.89261913e-05
   3.05255380e-05  2.94166657e-05]
 [-6.50705750e+02  5.30211639e+01 -4.92040396e+00 ...  4.75216802e-05
   3.46632551e-05  1.62844444e-05]
 ...
 [-5.50096191e+02  1.70297680e+01 -1.14575634e+01 ...  1.51764631e-04
   1.16828531e-04  8.47479314e-05]
 [-5.55357605e+02  4.71569710e+01  1.10750742e+01 ...  1.61086471e-04
   1.04962470e-04  6.52811723e-05]
 [-5.04816345e+02  3.53618660e+01 -1.43495779e+01 ...  6.08151546e-04
   5.55269653e-04  4.47782222e-04]] [[-6.01385986e+02  6.33781929e+01 -1.15760441e+01 ...  7.30556485e-06
   4.64438199e-06  1.78243295e-06]
 [-7.86435913e+02  5.99123230e+01  2.08771191e+01 ...  4.30930669e-07
   4.01358051e-07  4.06331111e-07]
 [-5.60618164e+02  5.78945961e+01 -8.87167645e+00 ...  6.46989676e-04
   3.31015413e-04  1.85833647e-04]
 ...
 [-4.44762360e+02  2.36132107e+01 -1.4835466

In [8]:
#shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [9]:
#number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [10]:
#initilize Multi layer perceptron classifier
model=MLPClassifier(alpha=0.01,batch_size=256,epsilon=1e-08,hidden_layer_sizes=(300,), learning_rate='adaptive',max_iter=1800)

In [11]:
#train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=1800)

In [12]:
#predict test set
y_pred=model.predict(x_test)

In [13]:
#accuracy of model
accuracy=accuracy_score(y_true=y_test,y_pred=y_pred)
print("accuracy:{:.2f}%".format(accuracy*100))

accuracy:72.40%


In [14]:
import pickle
model_filename="speech_emotion_recognition_model.pkl"
#save model to file
with open(model_filename,'wb') as file:
    pickle.dump(model,file)
print(f"Model saved to {model_filename}")

Model saved to speech_emotion_recognition_model.pkl


In [15]:
#load the model from file
with open(model_filename,'rb') as file:
    loaded_model=pickle.load(file)
print("Model Loaded success!")

Model Loaded success!


In [16]:
print("hi")

hi


In [20]:
import os
import librosa
import numpy as np
import pickle
import soundfile as sf

# Function to extract features
def extract_feature(file_name, mfcc=True, chroma=True, mel=True):
    with sf.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            stft = np.abs(librosa.stft(X))
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
            result = np.hstack((result, mel))
    return result

# Function to choose file and predict emotion
def choose_file_and_predict():
    file_path = input("Please enter the path to the audio file: ")

    if os.path.exists(file_path) and file_path.endswith('.wav'):
        # Extract features from the chosen file
        features = extract_feature(file_path, mfcc=True, chroma=True, mel=True)
        features = features.reshape(1, -1)

        # Load the trained model
        with open('speech_emotion_recognition_model.pkl', 'rb') as model_file:
            model = pickle.load(model_file)

        # Predict the emotion
        prediction = model.predict(features)
        print(f"The predicted emotion is: {prediction[0]}")
    else:
        print("Invalid file path or file format. Please provide a valid .wav file.")

# Call the function to choose a file and predict emotion
choose_file_and_predict()


Please enter the path to the audio file:  C:\Users\NERAJAKSHA\Desktop\New folder\ravdess data\Actor_17\03-01-05-02-02-01-17.wav


The predicted emotion is: fearful
